## Imports

In [31]:
import numpy as np
from scipy.sparse.linalg import spsolve
import igl
from meshplot import plot
import triangle

## Curves

In [35]:
class Curve:
    # p1 and p2 are tuples representing the start and end points
    # c1 and c2 are tuples representing the colors on each side
    def __init__(self, p1, p2, c1, c2):
        self.p1 = np.array(p1)
        self.p2 = np.array(p2)
        self.c1 = np.array(c1)
        self.c2 = np.array(c2)
    
    # Get n samples points on the curve
    def sample_points(self, n=100):
        points = []
        diff = self.p2 - self.p1
        for i in range(n):
            points.append(self.p1 + diff * i / (n-1))
        return np.array(points)


curves = np.array([
    Curve((0.25, 0.25, 0.0), (0.25, 0.75, 0.0), (1.0, 0.0, 0.0), (1.0, 0.0, 0.0)),
    Curve((0.75, 0.25, 0.0), (0.75, 0.75, 0.0), (0.0, 0.0, 1.0), (0.0, 0.0, 1.0)),
    Curve((0.75, 0.25, 0.0), (0.75, 0.75, 0.0), (0.0, 0.0, 1.0), (0.0, 0.0, 1.0))
])

## Setup

## Laplace

In [90]:
v_b = []
bc_red = []
bc_green = []
bc_blue = []

GRID_SIZE = 50
v = np.zeros(((GRID_SIZE+1)**2, 3))
i = 0
for x in range(GRID_SIZE+1):
    for y in range(GRID_SIZE+1):
        v[i] = x / GRID_SIZE, y / GRID_SIZE, 0.0
        if x == 0 or x == GRID_SIZE or y == 0 or y == GRID_SIZE:
            v_b.append(i)
            bc_red.append(1.0)
            bc_green.append(1.0)
            bc_blue.append(1.0)
        i += 1

vertex_i = v.shape[0]
for c in curves:
    points = c.sample_points(10)
    for i in range(points.shape[0]):
        v_b.append(vertex_i)
        bc_red.append(c.c1[0])
        bc_green.append(c.c1[1])
        bc_blue.append(c.c1[2])
        vertex_i += 1
    v = np.append(v, points, axis=0)

v_b = np.array(v_b)
triangulation = triangle.triangulate(dict(vertices=v[:,:2]))
f = triangulation["triangles"]

v_all = np.arange(v.shape[0])
v_in = np.setdiff1d(v_all, v_b)

l = igl.cotmatrix(v, f)
l_ii = l[v_in, :]
l_ii = l_ii[:, v_in]

l_ib = l[v_in, :]
l_ib = l_ib[:, v_b]

bc_red = np.array(bc_red)
bc_green = np.array(bc_green)
bc_blue = np.array(bc_blue)

red_in = spsolve(-l_ii, l_ib.dot(bc_red))
green_in = spsolve(-l_ii, l_ib.dot(bc_green))
blue_in = spsolve(-l_ii, l_ib.dot(bc_blue))

c = np.zeros(v.shape)
c[v_b, 0] = bc_red
c[v_b, 1] = bc_green
c[v_b, 2] = bc_blue
c[v_in, 0] = red_in
c[v_in, 1] = green_in
c[v_in, 2] = blue_in

## Plot

In [91]:
plot(v, f, c, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.5,…